# Important Modules

In [30]:
import sklearn
sklearn.__version__

'0.24.2'

In [1]:
import pandas as pd
import string
import re
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [3]:
import matplotlib.pyplot as plt

# Data Reading

In [4]:
df = pd.read_csv('./Language Detection.csv')


In [5]:
df

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English
...,...,...
11050,ನಿಮ್ಮ ತಪ್ಪು ಏನು ಬಂದಿದೆಯೆಂದರೆ ಆ ದಿನದಿಂದ ನಿಮಗೆ ಒ...,Kannada
11051,ನಾರ್ಸಿಸಾ ತಾನು ಮೊದಲಿಗೆ ಹೆಣಗಾಡುತ್ತಿದ್ದ ಮಾರ್ಗಗಳನ್...,Kannada
11052,ಹೇಗೆ ' ನಾರ್ಸಿಸಿಸಮ್ ಈಗ ಮರಿಯನ್ ಅವರಿಗೆ ಸಂಭವಿಸಿದ ಎ...,Kannada
11053,ಅವಳು ಈಗ ಹೆಚ್ಚು ಚಿನ್ನದ ಬ್ರೆಡ್ ಬಯಸುವುದಿಲ್ಲ ಎಂದು ...,Kannada


# Initial Insights of Data

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11055 entries, 0 to 11054
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      11055 non-null  object
 1   Language  11055 non-null  object
dtypes: object(2)
memory usage: 172.9+ KB


In [7]:
df.Language.value_counts()

English       1385
French        1014
Spanish        819
Hindi          781
Portugeese     739
Italian        698
Russian        692
Sweedish       676
Malayalam      594
Dutch          546
Arabic         536
Turkish        474
German         470
Tamil          469
Danish         428
Kannada        369
Greek          365
Name: Language, dtype: int64

In [8]:
df[df.Language == 'Russian'].sample(2)

,Text,Language
6784,В то время как Уэйлсу приписывается определени...,Russian
7375,"Я Терренс, ангел удачи.",Russian


In [9]:
df[df.Language == 'Malayalam'].sample(2)

,Text,Language
1522,ഒരേ സമയം വിക്കിസമൂഹത്തിന്റെ പടയാളികളും തൂപ്പുക...,Malayalam
1472,വിക്കിപീഡിയയുടെ വെബ് വിലാസത്തിന്റെ ആദ്യതാളിൽ വ...,Malayalam


In [10]:
df[df.Language == 'Arabic'].sample(2)

,Text,Language
10156,من دواعي سروري.,Arabic
9999,لا تأخذ الأمر على محمل الجد على محمل الجد لا ت...,Arabic


In [11]:
df[df.Language == 'Tamil'].sample(2)

,Text,Language
3181,நர்சிசா இந்த விதைகளை நடவு செய்ய நீங்கள் எனக்கு...,Tamil
3225,நர்சிசா அவள் முதலில் போராடிய வழிகளை மாற்றினாள்...,Tamil


In [12]:
df[df.Language == 'Kannada'].sample(2)

,Text,Language
11039,ಬೇರೇನಾದರೂ ಆದರೆ ನಾನು ತಿನ್ನಬಹುದಾದ ಸಾಮಾನ್ಯ ಬ್ರೆಡ್...,Kannada
10910,ನನ್ನನ್ನು ಸೋಲಿಸುತ್ತಾನೆ.,Kannada


In [13]:
df[df.Language == 'Hindi'].sample(2)

,Text,Language
2554,'गोलमाल अगेन' की स्टोरी: पांच लड़कों का ग्रुप-...,Hindi
2066,विश्व हिंदू परिषद के विरोधों के बीच थिअटर तक प...,Hindi


# Text Preprocessing

In [14]:
def removeSymbolsAndNumbers(text):        
        text = re.sub(r'[{}]'.format(string.punctuation), '', text)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[@]', '', text)

        return text.lower()

In [15]:
def removeEnglishLetters(text):        
        text = re.sub(r'[a-zA-Z]+', '', text)
        return text.lower()

In [16]:
X0 = df.apply(lambda x: removeEnglishLetters(x.Text) if x.Language in ['Russian','Malyalam','Hindi','Kannada','Tamil','Arabic']  else x.Text, axis = 1)
X0

0         Nature, in the broadest sense, is the natural...
1        "Nature" can refer to the phenomena of the phy...
2        The study of nature is a large, if not the onl...
3        Although humans are part of nature, human acti...
4        [1] The word nature is borrowed from the Old F...
                               ...                        
11050    ನಿಮ್ಮ ತಪ್ಪು ಏನು ಬಂದಿದೆಯೆಂದರೆ ಆ ದಿನದಿಂದ ನಿಮಗೆ ಒ...
11051    ನಾರ್ಸಿಸಾ ತಾನು ಮೊದಲಿಗೆ ಹೆಣಗಾಡುತ್ತಿದ್ದ ಮಾರ್ಗಗಳನ್...
11052    ಹೇಗೆ ' ನಾರ್ಸಿಸಿಸಮ್ ಈಗ ಮರಿಯನ್ ಅವರಿಗೆ ಸಂಭವಿಸಿದ ಎ...
11053    ಅವಳು ಈಗ ಹೆಚ್ಚು ಚಿನ್ನದ ಬ್ರೆಡ್ ಬಯಸುವುದಿಲ್ಲ ಎಂದು ...
11054    ಟೆರ್ರಿ ನೀವು ನಿಜವಾಗಿಯೂ ಆ ದೇವದೂತನಂತೆ ಸ್ವಲ್ಪ ಕಾಣು...
Length: 11055, dtype: object

In [17]:
X1 = X0.apply(removeSymbolsAndNumbers)
X1

0         nature in the broadest sense is the natural p...
1        nature can refer to the phenomena of the physi...
2        the study of nature is a large if not the only...
3        although humans are part of nature human activ...
4         the word nature is borrowed from the old fren...
                               ...                        
11050    ನಿಮ್ಮ ತಪ್ಪು ಏನು ಬಂದಿದೆಯೆಂದರೆ ಆ ದಿನದಿಂದ ನಿಮಗೆ ಒ...
11051    ನಾರ್ಸಿಸಾ ತಾನು ಮೊದಲಿಗೆ ಹೆಣಗಾಡುತ್ತಿದ್ದ ಮಾರ್ಗಗಳನ್...
11052    ಹೇಗೆ  ನಾರ್ಸಿಸಿಸಮ್ ಈಗ ಮರಿಯನ್ ಅವರಿಗೆ ಸಂಭವಿಸಿದ ಎಲ...
11053    ಅವಳು ಈಗ ಹೆಚ್ಚು ಚಿನ್ನದ ಬ್ರೆಡ್ ಬಯಸುವುದಿಲ್ಲ ಎಂದು ...
11054    ಟೆರ್ರಿ ನೀವು ನಿಜವಾಗಿಯೂ ಆ ದೇವದೂತನಂತೆ ಸ್ವಲ್ಪ ಕಾಣು...
Length: 11055, dtype: object

In [18]:
y = df['Language']

# Fitting Model

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X1,y, random_state=42)

In [20]:
vectorizer = TfidfVectorizer(ngram_range=(1,3), analyzer='char')

In [21]:
model = pipeline.Pipeline([
    ('vectorizer', vectorizer),
    ('clf', LogisticRegression())
])

In [22]:
model.fit(x_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('clf', LogisticRegression())])

# Checking for Accuracy 

In [23]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test,y_pred)

In [24]:
print("Accuracy is :",accuracy)

Accuracy is : 0.98589001447178


In [25]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

      Arabic       1.00      0.99      1.00       135
      Danish       0.97      0.96      0.96        96
       Dutch       0.96      0.96      0.96       136
     English       0.98      0.99      0.99       349
      French       0.99      0.98      0.98       253
      German       0.98      0.95      0.97       105
       Greek       1.00      1.00      1.00        89
       Hindi       1.00      1.00      1.00       181
     Italian       0.99      0.97      0.98       173
     Kannada       1.00      1.00      1.00        97
   Malayalam       1.00      0.99      1.00       156
  Portugeese       0.99      0.99      0.99       192
     Russian       1.00      0.99      1.00       186
     Spanish       0.97      0.98      0.98       196
    Sweedish       0.98      0.98      0.98       170
       Tamil       1.00      1.00      1.00       126
     Turkish       0.95      0.99      0.97       124

    accuracy              

# Testing Model 

In [31]:
def predict(text):
    lang = model.predict([text])
    print('This is in ',lang[0],' language')

In [32]:
# English
predict("LANGUAGE DETECTION MODEL CHECK")
# French
predict("VÉRIFICATION DU MODÈLE DE DÉTECTION DE LA LANGUE")
# Arabic
predict("توففحص نموذج الكشف عن اللغة")
# Spanish
predict("VERIFICACIÓN DEL MODELO DE DETECCIÓN DE IDIOMAS")
# Malayalam
predict("ലാംഗ്വേജ് ഡിറ്റക്ഷൻ മോഡൽ ചെക്ക്")
# Russian
predict("ПРОВЕРКА МОДЕЛИ ОПРЕДЕЛЕНИЯ ЯЗЫКА")
# Hindi
predict('भाषा का पता लगाने वाले मॉडल की जांच')
# Hindi
predict(' boyit9h एनालिटिक्स alhgserog 90980879809 bguytfivb ahgseporiga प्रदान करता है')
# Japanese
predict('言語検出モデルチェック')
# Turkish
predict('DİL TESPİT MODELİ KONTROLÜ')

This is in  English  language
This is in  French  language
This is in  Arabic  language
This is in  Spanish  language
This is in  Malayalam  language
This is in  Russian  language
This is in  Hindi  language
This is in  Hindi  language
This is in  Turkish  language
This is in  Turkish  language


In [28]:
# dump the information in pickle file
file = open('LDModel.pkl', 'wb')
pickle.dump(model, file)
file.close()